In [1]:
!pip install numpy>=1.20.0 pandas>=1.3.0 matplotlib>=3.4.0 seaborn>=0.11.0 scikit-learn>=1.0.0 tensorflow>=2.8.0 scikeras>=0.9.0 tqdm>=4.61.0

zsh:1: 1.20.0 not found


In [23]:
# # Insurance Charges Prediction Project (COSC 202)
#
# This notebook presents a comprehensive analysis and modeling of insurance charges prediction
# using neural networks. The goal is to predict insurance charges based on customer attributes
# such as age, BMI, smoking status, and other factors.

# Import necessary libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
import json
import pickle
import datetime
import time
from tqdm.auto import tqdm
from sklearn.model_selection import train_test_split, GridSearchCV, KFold
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.optimizers import Adam

# Use scikeras instead of the deprecated keras.wrappers.scikit_learn
try:
    from scikeras.wrappers import KerasRegressor
except ImportError:
    # Fallback for older TensorFlow versions
    try:
        from tensorflow.keras.wrappers.scikit_learn import KerasRegressor
    except ImportError:
        print("Warning: Could not import KerasRegressor. Please install scikeras package with: pip install scikeras")

# Set random seed for reproducibility
np.random.seed(42)
import tensorflow as tf
tf.random.set_seed(42)

In [24]:
# ## 1. Data Exploration
#
# This section explores the dataset to understand its structure, statistical properties,
# and identify potential issues (missing values, outliers).

def explore_data(df):
    """Explore the dataset and provide key statistics"""
    print("\n===== DATA EXPLORATION =====\n")

    print("Dataset Shape:", df.shape)
    print("\nData Types:")
    print(df.dtypes)

    print("\nBasic Statistics:")
    print(df.describe().T)

    print("\nChecking for Missing Values:")
    print(df.isnull().sum())

    # Check for categorical features
    categorical_cols = df.select_dtypes(include=['object']).columns
    print("\nCategorical Features:")
    for col in categorical_cols:
        print(f"\n{col} value counts:")
        print(df[col].value_counts())

    # Check for outliers in numerical features
    numerical_cols = df.select_dtypes(include=['int64', 'float64']).columns
    print("\nChecking for Outliers in Numerical Features:")
    for col in numerical_cols:
        Q1 = df[col].quantile(0.25)
        Q3 = df[col].quantile(0.75)
        IQR = Q3 - Q1
        outliers = df[(df[col] < Q1 - 1.5 * IQR) | (df[col] > Q3 + 1.5 * IQR)]
        print(f"{col}: {len(outliers)} outliers detected ({len(outliers)/len(df)*100:.2f}%)")

    return df


In [25]:
# ## 2. Data Visualization
#
# This section creates visualizations to understand feature relationships and their impact
# on insurance charges.

def visualize_data(df):
    """Visualize key patterns, correlations and distributions within dataset"""
    print("\n===== DATA VISUALIZATION =====\n")

    # Set figure aesthetics
    plt.style.use('seaborn-v0_8-whitegrid')
    sns.set(font_scale=1.2)

    # 1. Distribution of target variable (charges)
    plt.figure(figsize=(12, 6))
    plt.subplot(1, 2, 1)
    sns.histplot(df['charges'], kde=True)
    plt.title('Distribution of Insurance Charges')
    plt.xlabel('Charges ($)')
    plt.ylabel('Frequency')

    plt.subplot(1, 2, 2)
    sns.boxplot(y=df['charges'])
    plt.title('Boxplot of Insurance Charges')
    plt.ylabel('Charges ($)')
    plt.tight_layout()
    plt.savefig('charges_distribution.png')
    plt.close()

    # 2. Correlation matrix
    plt.figure(figsize=(10, 8))
    # Convert categorical variables to numeric for correlation
    df_corr = df.copy()
    df_corr['sex'] = df_corr['sex'].map({'female': 0, 'male': 1})
    df_corr['smoker'] = df_corr['smoker'].map({'no': 0, 'yes': 1})
    df_corr = pd.get_dummies(df_corr, columns=['region'], drop_first=True)

    corr = df_corr.corr()
    mask = np.triu(np.ones_like(corr, dtype=bool))
    sns.heatmap(corr, mask=mask, annot=True, fmt='.2f', cmap='coolwarm', square=True)
    plt.title('Correlation Matrix')
    plt.tight_layout()
    plt.savefig('correlation_matrix.png')
    plt.close()

    # 3. Relationship between numerical features and charges
    numerical_cols = ['age', 'bmi', 'children']
    plt.figure(figsize=(15, 5))
    for i, col in enumerate(numerical_cols):
        plt.subplot(1, 3, i+1)
        sns.scatterplot(x=col, y='charges', data=df, hue='smoker', alpha=0.7)
        plt.title(f'{col.capitalize()} vs Charges')
        plt.tight_layout()
    plt.savefig('numerical_vs_charges.png')
    plt.close()

    # 4. Relationship between categorical features and charges
    categorical_cols = ['sex', 'smoker', 'region']
    plt.figure(figsize=(15, 5))
    for i, col in enumerate(categorical_cols):
        plt.subplot(1, 3, i+1)
        sns.boxplot(x=col, y='charges', data=df)
        plt.title(f'{col.capitalize()} vs Charges')
        plt.tight_layout()
    plt.savefig('categorical_vs_charges.png')
    plt.close()

    # 5. BMI distribution by smoker status
    plt.figure(figsize=(10, 6))
    sns.kdeplot(data=df, x='bmi', hue='smoker', fill=True, common_norm=False)
    plt.title('BMI Distribution by Smoker Status')
    plt.xlabel('BMI')
    plt.ylabel('Density')
    plt.savefig('bmi_distribution.png')
    plt.close()

    # 6. Age vs Charges with BMI as size and smoker as hue
    plt.figure(figsize=(12, 8))
    sns.scatterplot(x='age', y='charges', size='bmi', hue='smoker', data=df, sizes=(20, 200), alpha=0.7)
    plt.title('Age vs Charges (Size: BMI, Color: Smoker Status)')
    plt.xlabel('Age')
    plt.ylabel('Charges ($)')
    plt.savefig('age_charges_bmi_smoker.png')
    plt.close()

    # Print visualization conclusion
    print("\nVisualization Conclusions:")
    print("1. The charges distribution is right-skewed, indicating most people have lower charges but a few have very high charges.")
    print("2. Smoking status shows the strongest correlation with charges, indicating smokers are charged significantly more.")
    print("3. Age shows a positive correlation with charges - as age increases, charges tend to increase.")
    print("4. BMI has a moderate positive correlation with charges, especially for smokers.")
    print("5. There appears to be an interaction effect between smoking and BMI - higher BMI smokers face the highest charges.")
    print("6. Region and sex have less impact on charges compared to smoking status, age, and BMI.")

    return df

In [26]:
# ## 3. Data Preprocessing
#
# This section handles data preprocessing including feature scaling, encoding categorical variables,
# and splitting data into training and testing sets.

def preprocess_data(df):
    """Preprocess the dataset for neural network modeling"""
    print("\n===== DATA PREPROCESSING =====\n")

    # Separate features and target
    X = df.drop('charges', axis=1)
    y = df['charges']

    # Identify categorical and numerical columns
    categorical_cols = X.select_dtypes(include=['object']).columns
    numerical_cols = X.select_dtypes(include=['int64', 'float64']).columns

    # Split the data into training and testing sets
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=0.2, random_state=42)

    # Save the original test data for later analysis
    X_test_original = X_test.copy()

    # Create transformers for preprocessing
    # Handle different versions of scikit-learn (sparse vs sparse_output parameter)
    try:
        # For scikit-learn 1.2+
        categorical_transformer = OneHotEncoder(drop='first', sparse_output=False)
    except TypeError:
        # For older scikit-learn versions
        categorical_transformer = OneHotEncoder(drop='first', sparse=False)

    numerical_transformer = StandardScaler()

    # Create a preprocessing pipeline using ColumnTransformer
    preprocessor = ColumnTransformer(
        transformers=[
            ('num', numerical_transformer, numerical_cols),
            ('cat', categorical_transformer, categorical_cols)
        ])

    # Fit the preprocessor on the training data
    X_train_processed = preprocessor.fit_transform(X_train)
    X_test_processed = preprocessor.transform(X_test)

    # Print preprocessing results
    print(f"Training set shape: {X_train_processed.shape}")
    print(f"Testing set shape: {X_test_processed.shape}")

    print("\nPreprocessing Steps Applied:")
    print("1. One-hot encoded categorical variables (sex, smoker, region) with drop='first' to avoid dummy variable trap")
    print("2. Standardized numerical features (age, bmi, children) to have zero mean and unit variance")
    print("3. Split data into 80% training and 20% testing sets")

    # Return the processed datasets, the preprocessor, and original test data
    return X_train_processed, X_test_processed, y_train, y_test, preprocessor, X_test_original


In [27]:
# ## 4. Neural Network Model Design
#
# This section defines the neural network architecture for predicting insurance charges.

def create_model(input_dim, hidden_layers=2, neurons=64, dropout_rate=0.2, learning_rate=0.001):
    """Create a neural network model for regression"""
    model = Sequential()

    # Input layer
    model.add(Dense(neurons, input_dim=input_dim, activation='relu'))
    model.add(Dropout(dropout_rate))

    # Hidden layers
    for _ in range(hidden_layers - 1):
        model.add(Dense(neurons, activation='relu'))
        model.add(Dropout(dropout_rate))

    # Output layer (linear activation for regression)
    model.add(Dense(1, activation='linear'))

    # Compile the model
    model.compile(
        optimizer=Adam(learning_rate=learning_rate),
        loss='mse',
        metrics=['mae']
    )

    return model


In [28]:
# ## 5. Hyperparameter Tuning
#
# This section performs grid search with cross-validation to find optimal model hyperparameters.

def tune_hyperparameters(X_train, y_train):
    """Tune neural network hyperparameters using GridSearchCV"""
    print("\n===== HYPERPARAMETER TUNING =====\n")

    # Define a simplified model function that works with both keras and scikeras wrappers
    def create_model_simple(input_dim=X_train.shape[1], hidden_layers=2, neurons=64, dropout_rate=0.2, learning_rate=0.001):
        model = Sequential()

        # Input layer
        model.add(Dense(neurons, input_dim=input_dim, activation='relu'))
        model.add(Dropout(dropout_rate))

        # Hidden layers
        for _ in range(hidden_layers - 1):
            model.add(Dense(neurons, activation='relu'))
            model.add(Dropout(dropout_rate))

        # Output layer (linear activation for regression)
        model.add(Dense(1, activation='linear'))

        # Compile the model
        model.compile(
            optimizer=Adam(learning_rate=learning_rate),
            loss='mse',
            metrics=['mae']
        )

        return model

    # Create KerasRegressor wrapper - handle both legacy and scikeras versions
    try:
        # Check if we're using scikeras version
        from scikeras.wrappers import KerasRegressor as ScikKerasRegressor
        print("Using scikeras.wrappers.KerasRegressor")

        # For scikeras, parameters are passed differently
        model = ScikKerasRegressor(
            model=create_model_simple,
            epochs=100,
            batch_size=32,
            verbose=0
        )

        # Define parameter grid for scikeras format
        param_grid = {
            'model__hidden_layers': [1, 2, 3],
            'model__neurons': [32, 64, 128],
            'model__dropout_rate': [0.1, 0.2, 0.3],
            'model__learning_rate': [0.01, 0.001, 0.0001],
            'batch_size': [16, 32, 64],
            'epochs': [50, 100]
        }
    except ImportError:
        # Legacy TensorFlow implementation
        print("Using legacy tensorflow.keras.wrappers.scikit_learn.KerasRegressor")
        from tensorflow.keras.wrappers.scikit_learn import KerasRegressor as TFKerasRegressor

        model = TFKerasRegressor(
            build_fn=create_model_simple,
            epochs=100,
            batch_size=32,
            verbose=0
        )

        # Legacy parameter grid
        param_grid = {
            'hidden_layers': [1, 2, 3],
            'neurons': [32, 64, 128],
            'dropout_rate': [0.1, 0.2, 0.3],
            'learning_rate': [0.01, 0.001, 0.0001],
            'batch_size': [16, 32, 64],
            'epochs': [50, 100]
        }

    # Define K-Fold cross-validation
    kfold = KFold(n_splits=5, shuffle=True, random_state=42)

    # Create GridSearchCV
    grid = GridSearchCV(
        estimator=model,
        param_grid=param_grid,
        cv=kfold,
        scoring='neg_mean_squared_error',
        n_jobs=-1  # Use all available cores
    )

    print("Starting hyperparameter tuning with 5-fold cross-validation...")
    print("This may take some time to complete.")

    # Calculate total combinations for progress reporting
    total_combinations = len(param_grid['batch_size']) * len(param_grid['epochs'])

    # Get hidden_layers param appropriately whether using scikeras or legacy
    if 'model__hidden_layers' in param_grid:
        total_combinations *= len(param_grid['model__hidden_layers'])
        total_combinations *= len(param_grid['model__neurons'])
        total_combinations *= len(param_grid['model__dropout_rate'])
        total_combinations *= len(param_grid['model__learning_rate'])
    else:
        total_combinations *= len(param_grid['hidden_layers'])
        total_combinations *= len(param_grid['neurons'])
        total_combinations *= len(param_grid['dropout_rate'])
        total_combinations *= len(param_grid['learning_rate'])

    # Calculate total fits (combinations * CV folds)
    total_fits = total_combinations * kfold.get_n_splits()

    print(f"\nPerforming {total_combinations} parameter combinations with {kfold.get_n_splits()}-fold cross-validation")
    print(f"Total number of model fits: {total_fits}")
    print("This process may take a while. Progress updates will be shown periodically.")

    # Set verbose to 1 to get some progress information from GridSearchCV
    grid.verbose = 1

    # Fit GridSearchCV
    print("\nStarting hyperparameter search...")
    start_time = time.time()
    grid_result = grid.fit(X_train, y_train)
    elapsed_time = time.time() - start_time

    # Print timing information
    hours, remainder = divmod(elapsed_time, 3600)
    minutes, seconds = divmod(remainder, 60)
    print(f"\nHyperparameter search completed in {int(hours):02d}:{int(minutes):02d}:{int(seconds):02d}")

    # Print results
    print(f"\nBest: {-grid_result.best_score_:.2f} MSE using {grid_result.best_params_}")

    # Print all results
    print("\nGrid Search Results:")
    means = -grid_result.cv_results_['mean_test_score']
    stds = grid_result.cv_results_['std_test_score']
    params = grid_result.cv_results_['params']

    # Display top 5 results
    top_indices = means.argsort()[:5]
    print("\nTop 5 hyperparameter combinations:")
    for i in top_indices:
        print(f"MSE: {means[i]:.2f} (+/- {stds[i]:.2f}) with: {params[i]}")

    # Return best parameters in standardized format (without model__ prefix)
    best_params = grid_result.best_params_.copy()

    # For scikeras, convert model__param to param for consistency
    if 'model__hidden_layers' in best_params:
        best_params = {
            'hidden_layers': best_params.get('model__hidden_layers', 2),
            'neurons': best_params.get('model__neurons', 64),
            'dropout_rate': best_params.get('model__dropout_rate', 0.2),
            'learning_rate': best_params.get('model__learning_rate', 0.001),
            'batch_size': best_params.get('batch_size', 32),
            'epochs': best_params.get('epochs', 100)
        }

    return best_params


In [29]:
# ## 6. Model Training and Evaluation
#
# This section trains the final model with the best hyperparameters and evaluates performance.

def train_and_evaluate(X_train, X_test, y_train, y_test, best_params, input_dim, X_test_original=None):
    """Train and evaluate the neural network model with best hyperparameters"""
    print("\n===== MODEL TRAINING AND EVALUATION =====\n")

    # Extract best parameters
    hidden_layers = best_params.get('hidden_layers', 2)
    neurons = best_params.get('neurons', 64)
    dropout_rate = best_params.get('dropout_rate', 0.2)
    learning_rate = best_params.get('learning_rate', 0.001)
    batch_size = best_params.get('batch_size', 32)
    epochs = best_params.get('epochs', 100)

    # Create model with best parameters
    model = create_model(
        input_dim=input_dim,
        hidden_layers=hidden_layers,
        neurons=neurons,
        dropout_rate=dropout_rate,
        learning_rate=learning_rate
    )

    # Create callbacks
    early_stopping = EarlyStopping(
        monitor='val_loss',
        patience=20,
        restore_best_weights=True
    )

    model_checkpoint = ModelCheckpoint(
        'best_model.h5',
        save_best_only=True,
        monitor='val_loss',
        mode='min'
    )

    # Create progress bar callback for training
    class TqdmProgressCallback(tf.keras.callbacks.Callback):
        def on_epoch_begin(self, epoch, logs=None):
            self.pbar = tqdm(total=int(self.params['steps']),
                            desc=f"Epoch {epoch+1}/{self.params['epochs']}",
                            position=0, leave=True)

        def on_batch_end(self, batch, logs=None):
            self.pbar.update(1)

        def on_epoch_end(self, epoch, logs=None):
            self.pbar.close()
            train_loss = logs.get('loss', 0)
            val_loss = logs.get('val_loss', 0)
            train_mae = logs.get('mae', 0)
            val_mae = logs.get('val_mae', 0)
            print(f"Epoch {epoch+1}/{self.params['epochs']} - loss: {train_loss:.4f} - val_loss: {val_loss:.4f} - mae: {train_mae:.4f} - val_mae: {val_mae:.4f}")

    # Train the model
    print("Training the final model with best hyperparameters...")
    history = model.fit(
        X_train,
        y_train,
        validation_split=0.2,
        epochs=epochs,
        batch_size=batch_size,
        verbose=0,  # Set to 0 since we're using our custom progress bar
        callbacks=[early_stopping, model_checkpoint, TqdmProgressCallback()]
    )

    # Evaluate the model on test data with a progress bar
    print("Evaluating model on test data...")
    y_pred = np.zeros((len(X_test), 1))

    # Use batch prediction with progress bar for large datasets
    batch_size = 32
    num_batches = int(np.ceil(len(X_test) / batch_size))

    for i in tqdm(range(num_batches), desc="Predicting"):
        start_idx = i * batch_size
        end_idx = min((i + 1) * batch_size, len(X_test))
        y_pred[start_idx:end_idx] = model.predict(X_test[start_idx:end_idx], verbose=0)

    # Calculate performance metrics
    mse = mean_squared_error(y_test, y_pred)
    rmse = np.sqrt(mse)
    mae = mean_absolute_error(y_test, y_pred)
    r2 = r2_score(y_test, y_pred)

    print("\nModel Performance on Test Data:")
    print(f"Mean Squared Error (MSE): {mse:.2f}")
    print(f"Root Mean Squared Error (RMSE): {rmse:.2f}")
    print(f"Mean Absolute Error (MAE): {mae:.2f}")
    print(f"R² Score: {r2:.4f}")

    # Enhanced visualization of training history
    plt.figure(figsize=(15, 10))

    # Plot 1: Loss curves
    plt.subplot(2, 2, 1)
    plt.plot(history.history['loss'], label='Training Loss', linewidth=2)
    plt.plot(history.history['val_loss'], label='Validation Loss', linewidth=2)
    plt.title('Loss Curves', fontsize=14)
    plt.xlabel('Epoch', fontsize=12)
    plt.ylabel('Loss (MSE)', fontsize=12)
    plt.grid(True, linestyle='--', alpha=0.7)
    plt.legend(fontsize=10)

    # Plot 2: MAE curves
    plt.subplot(2, 2, 2)
    plt.plot(history.history['mae'], label='Training MAE', color='green', linewidth=2)
    plt.plot(history.history['val_mae'], label='Validation MAE', color='darkgreen', linewidth=2)
    plt.title('Mean Absolute Error Curves', fontsize=14)
    plt.xlabel('Epoch', fontsize=12)
    plt.ylabel('MAE', fontsize=12)
    plt.grid(True, linestyle='--', alpha=0.7)
    plt.legend(fontsize=10)

    # Plot 3: Actual vs Predicted values
    plt.subplot(2, 2, 3)
    plt.scatter(y_test, y_pred, alpha=0.6, edgecolor='k')
    plt.plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], 'r--', linewidth=2)
    plt.xlabel('Actual Charges', fontsize=12)
    plt.ylabel('Predicted Charges', fontsize=12)
    plt.title('Actual vs Predicted Insurance Charges', fontsize=14)
    plt.grid(True, linestyle='--', alpha=0.7)

    # Plot 4: Prediction Error Distribution
    plt.subplot(2, 2, 4)
    errors = y_test - y_pred.flatten()
    sns.histplot(errors, kde=True)
    plt.axvline(x=0, color='r', linestyle='--')
    plt.title('Prediction Error Distribution', fontsize=14)
    plt.xlabel('Prediction Error', fontsize=12)
    plt.ylabel('Frequency', fontsize=12)
    plt.grid(True, linestyle='--', alpha=0.7)

    plt.tight_layout()
    plt.savefig('training_evaluation.png', dpi=300, bbox_inches='tight')

    # Additional plots for model analysis
    plt.figure(figsize=(15, 12))

    # Plot 1: Residuals vs Predicted Values
    plt.subplot(2, 2, 1)
    plt.scatter(y_pred, errors, alpha=0.6, edgecolor='k')
    plt.axhline(y=0, color='r', linestyle='--', linewidth=2)
    plt.title('Residuals vs Predicted Values', fontsize=14)
    plt.xlabel('Predicted Values', fontsize=12)
    plt.ylabel('Residuals', fontsize=12)
    plt.grid(True, linestyle='--', alpha=0.7)

    # Plot 2: Prediction Error vs Age
    if 'age' in X_test_original.columns:
        plt.subplot(2, 2, 2)
        plt.scatter(X_test_original['age'], errors, alpha=0.6, edgecolor='k')
        plt.axhline(y=0, color='r', linestyle='--', linewidth=2)
        plt.title('Prediction Error vs Age', fontsize=14)
        plt.xlabel('Age', fontsize=12)
        plt.ylabel('Prediction Error', fontsize=12)
        plt.grid(True, linestyle='--', alpha=0.7)

    # Plot 3: Prediction Error vs BMI
    if 'bmi' in X_test_original.columns:
        plt.subplot(2, 2, 3)
        plt.scatter(X_test_original['bmi'], errors, alpha=0.6, edgecolor='k')
        plt.axhline(y=0, color='r', linestyle='--', linewidth=2)
        plt.title('Prediction Error vs BMI', fontsize=14)
        plt.xlabel('BMI', fontsize=12)
        plt.ylabel('Prediction Error', fontsize=12)
        plt.grid(True, linestyle='--', alpha=0.7)

    # Plot 4: Error Distribution by Smoker Status
    if 'smoker' in X_test_original.columns:
        plt.subplot(2, 2, 4)
        sns.boxplot(x=X_test_original['smoker'], y=errors)
        plt.axhline(y=0, color='r', linestyle='--', linewidth=2)
        plt.title('Prediction Error by Smoker Status', fontsize=14)
        plt.xlabel('Smoker', fontsize=12)
        plt.ylabel('Prediction Error', fontsize=12)
        plt.grid(True, linestyle='--', alpha=0.7)

    plt.tight_layout()
    plt.savefig('model_analysis.png', dpi=300, bbox_inches='tight')

    return model, history, (mse, rmse, mae, r2)


In [30]:
# ## 7. Model Saving and Persistence
#
# This section implements functionality to save all model artifacts and reload them later.

def save_model_artifacts(model, preprocessor, history, metrics, best_params, output_dir='model_artifacts'):
    """
    Save all model artifacts including the trained model, preprocessor, history, metrics, and parameters.

    Parameters:
    -----------
    model : tensorflow.keras.Model
        The trained neural network model
    preprocessor : sklearn.compose.ColumnTransformer
        The preprocessing pipeline used for feature transformation
    history : tensorflow.keras.callbacks.History
        Training history object containing loss and metrics
    metrics : tuple
        Tuple containing (mse, rmse, mae, r2) performance metrics
    best_params : dict
        Dictionary of best hyperparameters
    output_dir : str, default='model_artifacts'
        Directory to save model artifacts
    """
    # Create output directory if it doesn't exist
    os.makedirs(output_dir, exist_ok=True)

    # Generate timestamp for versioning
    timestamp = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")

    # Define the artifacts to save
    artifacts = [
        "Model",
        "Preprocessor",
        "Training History",
        "Performance Metrics",
        "Hyperparameters",
        "Manifest"
    ]

    # Create progress bar for saving artifacts
    with tqdm(total=len(artifacts), desc="Saving Model Artifacts") as saving_pbar:
        # 1. Save the trained model
        model_path = os.path.join(output_dir, f"insurance_model_{timestamp}.h5")
        model.save(model_path)
        saving_pbar.update(1)
        saving_pbar.set_description(f"Saved {artifacts[0]}")

        # 2. Save the preprocessor
        preprocessor_path = os.path.join(output_dir, f"preprocessor_{timestamp}.pkl")
        with open(preprocessor_path, 'wb') as f:
            pickle.dump(preprocessor, f)
        saving_pbar.update(1)
        saving_pbar.set_description(f"Saved {artifacts[1]}")

        # 3. Save training history
        history_path = os.path.join(output_dir, f"training_history_{timestamp}.json")
        with open(history_path, 'w') as f:
            history_dict = {key: [float(val) for val in values] for key, values in history.history.items()}
            json.dump(history_dict, f)
        saving_pbar.update(1)
        saving_pbar.set_description(f"Saved {artifacts[2]}")

        # 4. Save performance metrics
        metrics_path = os.path.join(output_dir, f"metrics_{timestamp}.json")
        mse, rmse, mae, r2 = metrics
        metrics_dict = {
            'mse': float(mse),
            'rmse': float(rmse),
            'mae': float(mae),
            'r2': float(r2)
        }
        with open(metrics_path, 'w') as f:
            json.dump(metrics_dict, f)
        saving_pbar.update(1)
        saving_pbar.set_description(f"Saved {artifacts[3]}")

        # 5. Save hyperparameters
        params_path = os.path.join(output_dir, f"hyperparameters_{timestamp}.json")
        with open(params_path, 'w') as f:
            json.dump(best_params, f)
        saving_pbar.update(1)
        saving_pbar.set_description(f"Saved {artifacts[4]}")

        # 6. Save a manifest file with paths to all artifacts
        manifest_path = os.path.join(output_dir, f"manifest_{timestamp}.json")
        manifest = {
            'model_path': model_path,
            'preprocessor_path': preprocessor_path,
            'history_path': history_path,
            'metrics_path': metrics_path,
            'params_path': params_path,
            'timestamp': timestamp
        }
        with open(manifest_path, 'w') as f:
            json.dump(manifest, f)
        saving_pbar.update(1)
        saving_pbar.set_description(f"Saved {artifacts[5]}")

    # Print summary of saved artifacts
    print(f"\nAll model artifacts saved successfully with timestamp: {timestamp}")
    print(f"  - Model: {os.path.basename(model_path)}")
    print(f"  - Preprocessor: {os.path.basename(preprocessor_path)}")
    print(f"  - History: {os.path.basename(history_path)}")
    print(f"  - Metrics: {os.path.basename(metrics_path)}")
    print(f"  - Parameters: {os.path.basename(params_path)}")
    print(f"  - Manifest: {os.path.basename(manifest_path)}")

    return manifest


def load_model_artifacts(manifest_path=None, timestamp=None, output_dir='model_artifacts'):
    """
    Load model artifacts from saved files.

    Parameters:
    -----------
    manifest_path : str, optional
        Path to the manifest file. If provided, will load artifacts specified in the manifest.
    timestamp : str, optional
        Timestamp of the artifacts to load. If provided instead of manifest_path, will try to load
        artifacts with this timestamp.
    output_dir : str, default='model_artifacts'
        Directory where model artifacts are saved

    Returns:
    --------
    model : tensorflow.keras.Model
        The loaded model
    preprocessor : sklearn.compose.ColumnTransformer
        The loaded preprocessor
    metrics : dict
        Dictionary containing the performance metrics
    params : dict
        Dictionary containing the hyperparameters
    """
    # If manifest path is provided, load from manifest
    if manifest_path and os.path.exists(manifest_path):
        with open(manifest_path, 'r') as f:
            manifest = json.load(f)
            model_path = manifest['model_path']
            preprocessor_path = manifest['preprocessor_path']
            metrics_path = manifest['metrics_path']
            params_path = manifest['params_path']
    # If timestamp is provided, construct paths
    elif timestamp:
        model_path = os.path.join(output_dir, f"insurance_model_{timestamp}.h5")
        preprocessor_path = os.path.join(output_dir, f"preprocessor_{timestamp}.pkl")
        metrics_path = os.path.join(output_dir, f"metrics_{timestamp}.json")
        params_path = os.path.join(output_dir, f"hyperparameters_{timestamp}.json")
    else:
        # Find the most recent manifest file
        manifest_files = [f for f in os.listdir(output_dir) if f.startswith('manifest_')]
        if not manifest_files:
            raise FileNotFoundError("No manifest files found in the output directory.")

        latest_manifest = sorted(manifest_files)[-1]
        manifest_path = os.path.join(output_dir, latest_manifest)

        with open(manifest_path, 'r') as f:
            manifest = json.load(f)
            model_path = manifest['model_path']
            preprocessor_path = manifest['preprocessor_path']
            metrics_path = manifest['metrics_path']
            params_path = manifest['params_path']

    # Load model
    print(f"Loading model from {model_path}...")
    model = load_model(model_path)

    # Load preprocessor
    print(f"Loading preprocessor from {preprocessor_path}...")
    with open(preprocessor_path, 'rb') as f:
        preprocessor = pickle.load(f)

    # Load metrics
    print(f"Loading metrics from {metrics_path}...")
    with open(metrics_path, 'r') as f:
        metrics = json.load(f)

    # Load hyperparameters
    print(f"Loading hyperparameters from {params_path}...")
    with open(params_path, 'r') as f:
        params = json.load(f)

    return model, preprocessor, metrics, params


def make_prediction(model, preprocessor, new_data):
    """
    Make predictions on new data using the loaded model and preprocessor.

    Parameters:
    -----------
    model : tensorflow.keras.Model
        The trained model
    preprocessor : sklearn.compose.ColumnTransformer
        The preprocessing pipeline
    new_data : pandas.DataFrame
        New data to make predictions on

    Returns:
    --------
    predictions : numpy.ndarray
        Model predictions
    """
    # Preprocess the new data
    new_data_processed = preprocessor.transform(new_data)

    # Make predictions
    predictions = model.predict(new_data_processed)

    return predictions


In [32]:
# Main execution
if __name__ == "__main__":
    # Show progress bar with project stages
    project_steps = [
        "Loading dataset",
        "Data exploration",
        "Data visualization",
        "Data preprocessing",
        "Hyperparameter tuning",
        "Model training",
        "Model evaluation",
        "Saving model artifacts"
    ]

    with tqdm(total=len(project_steps), desc="Project Progress", position=0) as project_progress:
        # Load the dataset
        print(f"\n{project_steps[0]}...")
        df = pd.read_csv('insurance.csv')
        project_progress.update(1)

        # Data exploration
        print(f"\n{project_steps[1]}...")
        df = explore_data(df)
        project_progress.update(1)

        # Data visualization
        print(f"\n{project_steps[2]}...")
        df = visualize_data(df)
        project_progress.update(1)

        # Preprocess the data
        print(f"\n{project_steps[3]}...")
        X_train_processed, X_test_processed, y_train, y_test, preprocessor, X_test_original = preprocess_data(df)
        project_progress.update(1)

    # Get number of input features after preprocessing
    input_dim = X_train_processed.shape[1]

    # Perform hyperparameter tuning
    # Comment out for faster execution during development
    print(f"\n{project_steps[4]}...")
    best_params = tune_hyperparameters(X_train_processed, y_train)
    project_progress.update(1)

    # If you want to skip hyperparameter tuning, uncomment and use these parameters instead
    # best_params = {
    #     'hidden_layers': 2,
    #     'neurons': 64,
    #     'dropout_rate': 0.2,
    #     'learning_rate': 0.001,
    #     'batch_size': 32,
    #     'epochs': 100
    # }

    # Train and evaluate the model
    print(f"\n{project_steps[5]} and {project_steps[6]}...")
    model, history, metrics = train_and_evaluate(
        X_train_processed, X_test_processed, y_train, y_test, best_params, input_dim, X_test_original)
    project_progress.update(1)  # Count both training and evaluation as one step

    # Save all model artifacts
    print("\nSaving model artifacts...")
    with tqdm(total=5, desc="Saving artifacts") as saving_progress:
        manifest = save_model_artifacts(model, preprocessor, history, metrics, best_params)
        saving_progress.update(5)  # Mark all saving steps complete
    project_progress.update(1)

    # Example of how to load the saved model and make predictions
    print("\n===== DEMONSTRATING MODEL LOADING =====\n")
    loaded_model, loaded_preprocessor, loaded_metrics, loaded_params = load_model_artifacts(
        manifest_path=manifest['manifest_path'])

    # Create a small sample for prediction demonstration
    sample = df.iloc[:5, :-1]  # First 5 rows, all columns except target
    predictions = make_prediction(loaded_model, loaded_preprocessor, sample)

    print("\nSample Predictions:")
    for i, (idx, row) in enumerate(sample.iterrows()):
        print(f"Sample {i+1}: Age: {row['age']}, Sex: {row['sex']}, BMI: {row['bmi']:.1f}, "
              f"Children: {row['children']}, Smoker: {row['smoker']}, Region: {row['region']}")
        print(f"Predicted Charge: ${predictions[i][0]:.2f}, Actual Charge: ${df.iloc[i]['charges']:.2f}\n")

    print("\n===== PROJECT SUMMARY =====\n")
    print("1. Explored the insurance dataset and identified key patterns")
    print("2. Visualized relationships between features and insurance charges")
    print("3. Preprocessed the data by encoding categorical variables and scaling numerical features")
    print("4. Performed hyperparameter tuning with cross-validation to find optimal neural network configuration")
    print("5. Trained final neural network model with early stopping to prevent overfitting")
    print("6. Evaluated model performance using multiple metrics")
    print("7. Saved all model artifacts for future use (model, preprocessor, metrics, hyperparameters)")
    print("8. Implemented functionality to load saved models and make new predictions")
    print("\nThe model can now be used to predict insurance charges based on customer attributes.")
    print("All visualizations and model artifacts have been saved in the current directory.")

Project Progress:   0%|          | 0/8 [00:00<?, ?it/s]


Loading dataset...

Data exploration...

===== DATA EXPLORATION =====

Dataset Shape: (1338, 7)

Data Types:
age           int64
sex          object
bmi         float64
children      int64
smoker       object
region       object
charges     float64
dtype: object

Basic Statistics:
           count          mean           std        min         25%       50%  \
age       1338.0     39.207025     14.049960    18.0000    27.00000    39.000   
bmi       1338.0     30.663397      6.098187    15.9600    26.29625    30.400   
children  1338.0      1.094918      1.205493     0.0000     0.00000     1.000   
charges   1338.0  13270.422265  12110.011237  1121.8739  4740.28715  9382.033   

                   75%          max  
age          51.000000     64.00000  
bmi          34.693750     53.13000  
children      2.000000      5.00000  
charges   16639.912515  63770.42801  

Checking for Missing Values:
age         0
sex         0
bmi         0
children    0
smoker      0
region      0
charges

KeyboardInterrupt: 

<Figure size 1500x500 with 0 Axes>